### DetNet

In [1]:
from torch import nn
import torch

In [2]:
class DetBottleneck(nn.Module):
    '''
    extra关键字，为False时为Bottleneck A ，为 True 时则为 Bottleneck B
    '''
    def __init__(self, inplanes, planes, stride=1, extra=False):
        super(DetBottleneck, self).__init__()

        # 连续构建3个卷积层
        self.bottleneck = nn.Sequential(
            nn.Conv2d(inplanes, planes, 1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(planes,
                      planes,
                      kernel_size=3,
                      stride=1,
                      padding=2,
                      dilation=2,
                      bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(planes, planes, 1, bias=False),
            nn.BatchNorm2d(planes),
        )
        self.relu = nn.ReLU(inplace=True)
        self.extra = extra

        # Bottleneck B 对应的 1X1 卷积
        if self.extra:
            self.extra_conv = nn.Sequential(
                nn.Conv2d(inplanes, planes, 1, bias=False),
                nn.BatchNorm2d(planes))

    def forward(self, x):
        if self.extra:
            identity = self.extra_conv(x)
        else:
            identity = x
        out = self.bottleneck(x)
        out += identity
        out = self.relu(out)
        return out

In [3]:
'''
构建一个 Stage 5 B-A-A结构
'''

bottleneck_b = DetBottleneck(1024, 256, 1, True)

In [4]:
bottleneck_b

DetBottleneck(
  (bottleneck): Sequential(
    (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace)
  (extra_conv): Sequential(
    (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [5]:
bottleneck_a1 = DetBottleneck(256, 256)

In [6]:
bottleneck_a2 = DetBottleneck(256, 256)

In [7]:
x=torch.randn(1,1024,14,14)
y1=bottleneck_b(x)
y2=bottleneck_a1(y1)
y_out=bottleneck_a2(y2)
y_out.shape

torch.Size([1, 256, 14, 14])